In [1]:
# Instalación de librerías necesarias
!pip install asyncpraw
!pip install praw
!pip install python-dotenv
!pip install scalene

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.0 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

# Modelo usado  ## TODO: Hacer pruebas con otros modelos o incluso dar la elección
emotion_model_path = "ayoubkirouane/BERT-Emotions-Classifier"

emotion_classifier = pipeline("text-classification", model=emotion_model_path, return_all_scores=True, padding=True, truncation=True,device=0)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [21]:
import asyncpraw
from dotenv import load_dotenv
import os
# Leer el archivo de entorno con las credenciales
load_dotenv("credencialesPraw.env")

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user = os.getenv("REDDIT_USER_AGENT")

reddit = asyncpraw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user)


In [50]:
## Obtención de los comentarios del Post elegido
#from praw.models import Comment
url = "https://www.reddit.com/r/law/comments/1kz6v7u/trump_wins_supreme_court_battle_to_deport_500000/"
submission = await reddit.submission(url=url)


comments = await submission.comments()

await comments.replace_more(limit=10)

comentarios = await comments.list()



In [51]:
# Función que dada una lista de comentarios, aplicamos el clasificador y nos quedamos la emoción más probable
def annotate_comments(comment_list,classifier=emotion_classifier):
    texts = [x.body for x in comment_list]
    results = classifier(texts)
    comments_dict = {}
    for comment, result in zip(comment_list, results):
        top = max(result, key=lambda x: x['score'])
        comments_dict[comment.id] = top['label']
        #comment['emotion_probs'] = result
    return comments_dict

# Recorremos los subreddits para aplicarle la función
comments_dict = annotate_comments(comentarios)


In [52]:
import polars as pl
# Uso de polars para agrupar por emocio y contar valores
polar = pl.DataFrame({'Comentario': list(comments_dict.keys()), 'Emoción': list(comments_dict.values())})
polar.group_by('Emoción').count()

<ipython-input-52-4b002ed7a1f1>:4: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  polar.group_by('Emoción').count()


Emoción,count
str,u32
"""anger""",423
"""optimism""",71
"""joy""",34
"""anticipation""",57
"""disgust""",94
"""fear""",19
"""sadness""",43
